# Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A **credit scoring** is used to evaluate the ability of a potential borrower to repay their loan.

## Open the data file and have a look at the general information. 

In [1]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer

In [2]:
scoring = pd.read_csv('../../datasets/credit_scoring_eng.csv')
scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
scoring.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [4]:
display(scoring.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


### Conclusion

We have missing values in columns 'days_employed' and 'total_income'. Also, it looks like there are some artifacts in 'days_employed'. Finally, values in 'education' and 'purpose' columns need to be unified.

## Data preprocessing

### Processing missing values

In [5]:
# -1 and 20 'children' are faulty values without any apparent pattern. 
# We have 47 and 76 such values, respectively (0.2 and 0.3 percent of the dataset).
# Let's change them to a mode value that is zero.
scoring.loc[scoring['children'] == -1, 'children'] = 0
scoring.loc[scoring['children'] == 20, 'children'] = 0

In [6]:
# 'days_employed' has missing values, and other values don't make much sense. 
# They are either negative or too large to be days (more than 900 years).
# Since this column has no connection to our questions, we could leave it as it is, 
# but consider notifying the person in charge of exporting the data.

In [7]:
# We have 101 zero values in 'dob_years' (0.4%). 
# Let's fill it with the mean (which is pretty close to the median here)

scoring.loc[scoring['dob_years'] == 0, 'dob_years'] = scoring['dob_years'].mean()

In [8]:
# The 'gender' column looks a bit strange.
# We have only one nonbinary value there, 
# while there should be at least 0.2-0.4% (43-86 minimum for this dataset).
# And this value is not nb or other, rather XNA.
# Let's drop it for now, but we need to check if our processes are inclusive.

scoring = scoring.drop(scoring[scoring['gender'] == 'XNA'].index).reset_index(drop=True)

In [9]:
# There are missing values in the 'total_income'.
# They correspond to the missing ones in 'days_employed', which we left untouched.
# But these are crucial for our analysis. 
# So let's fill them with median according to 'income_type' and 'education_id'.

scoring['total_income_cleaned'] = scoring['total_income'].fillna(
                            scoring.groupby(['income_type','education_id'])['total_income'].transform('median')
                            )

### Conclusion

Our dataset has 10% missing values in the 'total_income' column, which is essential for answering our questions. They are missing across all income types and other parameters without any visible pattern. So I filled them in a separate column with a median value for corresponding 'income_type'.

Also, for the same rows 'days_employed' value is missing, which I left as it is. All other values in this column don't look like days of employment also. They are either negative but close in absolute values to what could be reality. Or they are enormous for "retiree" and "unemployed" 'income_type's. We need to discuss the export of this column with data engineers.

Then there were 0.4% zero values in 'dob_years', which I filled with a mean value.

And finally, I found a small number of artifacts in 'children' (filled with mode) and 'gender' (dropped this one row).

### Data type replacement

In [10]:
scoring['dob_years'] = scoring['dob_years'].astype('int')
scoring['total_income_cleaned'] = scoring['total_income_cleaned'].astype('int')

### Conclusion

Here I converted 'dob_years' and 'filled_income' to int64 using Series.astype() method. 'days_employed' and 'total_income' left float since I won't use them later.

### Processing duplicates

In [11]:
#scoring.duplicated().sum()/len(scoring)
scoring = scoring.drop_duplicates().reset_index(drop=True)

### Conclusion

We have 0.25% of duplicated rows. It doesn't look like there is any pattern in them, deleted with the drop_duplicates method.

### Categorizing Data

In [12]:
# tidying up 'education' by making it lowercase

scoring['education'] = scoring['education'].str.lower()

In [13]:
# 'purpose' column has a lot of different values, which we could group into four categories: 
# wedding, property, buying a car, and getting an education.
wordnet_lemma = WordNetLemmatizer()
wedding_categ = ['wedding']
housing_categ = ['estate', 'housing', 'house', 'property']
car_categ = ['car']
education_categ = ['education', 'educated', 'university']

def lemmatization(s):
    words = nltk.word_tokenize(s.lower())
    lemmas = [wordnet_lemma.lemmatize(w) for w in words]
    if any(word in lemmas for word in wedding_categ):
        return 'wedding'
    elif any(word in lemmas for word in housing_categ):
        return 'property'
    elif any(word in lemmas for word in car_categ):
        return 'car'
    elif any(word in lemmas for word in education_categ):
        return 'education'
    else:
        return 'other'
    
scoring['purpose_cleaned']=scoring['purpose'].apply(lemmatization)    

In [14]:
# To answer a question about the relation of income to repaying a loan, 
# we need to split 'income_level' into several categories. 
# For example, it could be done by quartiles. 

qlabels = ['low', 'low medium', 'high medium', 'hight']
scoring['income_level'] = pd.qcut(scoring['total_income_cleaned'], q=4, labels=qlabels)

### Conclusion

I introduced an 'income_level ' category based on 'total_income_cleaned' and made a proper dictionary from the 'purpose' column. 

## Answer these questions

### Is there a relation between having kids and repaying a loan on time?

In [15]:
children=pd.pivot_table(scoring, index='children', 
                             values='debt', aggfunc=['sum', 'count'])
children['default_rate']=(children['sum']/children['count'])*100
children.sort_values('default_rate')

,sum,count,default_rate
,debt,debt,
children,,,
5,0,9,0.000000
0,1072,14229,7.533910
3,27,330,8.181818
1,444,4809,9.232689
2,194,2052,9.454191
4,4,41,9.756098


### Conclusion

We have zero defaults in people with five kids, but we can't conclude they are the best borrowers since we have only nine such data points. Childless people have the lowest default rates (7.5%), and among others, there is no correlation, all rates between 8.2% (3 kids) and 9.6% (4 kids).

### Is there a relation between marital status and repaying a loan on time?

In [16]:
family_status=pd.pivot_table(scoring, index='family_status', 
                             values='debt', aggfunc=['sum', 'count'])
family_status['default_rate']=(family_status['sum']/family_status['count'])*100
family_status.sort_values('default_rate')

,sum,count,default_rate
,debt,debt,
family_status,,,
widow / widower,63,959,6.569343
divorced,85,1195,7.112971
married,931,12344,7.542126
civil partnership,388,4162,9.322441
unmarried,274,2810,9.750890


In [17]:
def age_grouping(age):
    if age < 30:
        return '19-29'
    elif age < 40:
        return '30-39'
    elif age < 50:
        return '40-49'
    elif age < 60:
        return '50-59'
    else:
        return '60+'

scoring['age_group'] = scoring['dob_years'].apply(age_grouping)

age_group=pd.pivot_table(scoring, index='age_group', 
                             values='debt', aggfunc=['sum', 'count'])
age_group['default_rate']=(age_group['sum']/age_group['count'])*100
age_group.sort_values('default_rate')

,sum,count,default_rate
,debt,debt,
age_group,,,
60+,123,2506,4.908220
50-59,305,4662,6.542257
40-49,412,5459,7.547170
30-39,552,5663,9.747484
19-29,349,3180,10.974843


In [18]:
family_age_group=pd.pivot_table(scoring, index=['age_group', 'family_status'],
                             values='debt', aggfunc=['sum', 'count'])
family_age_group['default_rate']=(family_age_group['sum']/family_age_group['count'])*100
family_age_group.sort_values('default_rate')

sum count default_rate
                            debt  debt             
age_group family_status                            
19-29     widow / widower      0     2     0.000000
60+       unmarried            9   240     3.750000
          married             58  1310     4.427481
          civil partnership   18   399     4.511278
40-49     divorced            21   380     5.526316
50-59     widow / widower     21   376     5.585106
          married            168  2765     6.075949
          divorced            19   303     6.270627
60+       widow / widower     27   402     6.716418
          divorced            11   155     7.096774
40-49     married            235  3295     7.132018
30-39     widow / widower      3    41     7.317073
50-59     unmarried           31   413     7.506053
30-39     divorced            20   261     7.662835
50-59     civil partnership   66   805     8.198758
40-49     widow / widower     12   138     8.695652
          unmarried           47   539     8.719852
          civil partnership   97  1107     8.762421
30-39     married            308  3493     8.817635
19-29     civil partnership   70   684    10.233918
          married            162  1481    10.938555
          unmarried          103   917    11.232279
30-39     civil partnership  137  1167    11.739503
          unmarried           84   701    11.982882
19-29     divorced            14    96    14.583333

### Conclusion

The lowest default rates are among widowers and the highest among unmarried persons (with civil partnership status next). 

This observation leads me to a question if there is any correlation between age and default rate. And indeed, the highest default rates are among people younger than 30 and the lowest in the 60+ cohort.

Let's try to group our data by family status and age. We see that the most unreliable are divorced people in their 20s (constantly changing their mind on everything?), then unmarried and in civil partnership people 30-39 y.o. and then all other in 19-29. They all have default rates of more than 10%.
For married people in their 30s and not married people in their 40s (widowers, unmarried, c/p), we have rated between 8.7-8.8%. Next, we have 50-59 y.o. c/p with the rate of 8.2%. All other groups are following.

In [19]:
pd.pivot_table(scoring,index=['family_status'],values=['debt'], columns=['age_group'], aggfunc=['mean'])

mean                                        
                       debt                                        
age_group             19-29     30-39     40-49     50-59       60+
family_status                                                      
civil partnership  0.102339  0.117395  0.087624  0.081988  0.045113
divorced           0.145833  0.076628  0.055263  0.062706  0.070968
married            0.109386  0.088176  0.071320  0.060759  0.044275
unmarried          0.112323  0.119829  0.087199  0.075061  0.037500
widow / widower    0.000000  0.073171  0.086957  0.055851  0.067164

### Is there a relation between income level and repaying a loan on time?

In [20]:
income_level=pd.pivot_table(scoring, index='income_level', 
                             values='debt', aggfunc=['sum', 'count'])
income_level['default_rate']=(income_level['sum']/income_level['count'])*100
income_level.sort_values('default_rate')

,sum,count,default_rate
,debt,debt,
income_level,,,
hight,374,5368,6.967213
low,427,5368,7.954545
high medium,456,5367,8.496367
low medium,484,5367,9.018073


### Conclusion

The top 25% of incomes have the lowest default rates. Other than that, there is no correlation between income level and repaying a loan. (I also tried to split incomes into 10 and 15 equal baskets, but it showed nothing.) 

### How do different loan purposes affect on-time repayment of the loan?

In [21]:
purpose_cleaned=pd.pivot_table(scoring, index='purpose_cleaned', 
                             values='debt', aggfunc=['sum', 'count'])
purpose_cleaned['default_rate']=(purpose_cleaned['sum']/purpose_cleaned['count'])*100
purpose_cleaned.sort_values('default_rate')

,sum,count,default_rate
,debt,debt,
purpose_cleaned,,,
property,782,10813,7.232036
wedding,186,2335,7.965739
education,370,4014,9.217738
car,403,4308,9.354689


### Conclusion

The safest loan purposes are related to the property (7.2% default rate). And the riskiest ones are buying a car (9.3%) and education (9.2%)

## General conclusion

We have some data about past loans. We could use it to determine if a customer's marital status and number of children impact whether they will default on a loan. 

From these data, we don't see that customers' marital status influences whether they will repay. Instead, we could make predictions based on a person's age (The older is the customer, the safer is the loan). 

Also, having any number of children negatively affects the chance of default.

There is no correlation between income and the chance of repaying.

We could make predictions based on the purpose of the loan. For example, all purposes related to real estate are safe while getting an education or buying a car are the riskiest.